# Make plots to compare effective area and resolving power of Lynx XGS with other missiosn

## Chandra data

I selected some examples in Chaser, based on typical usage scenarios.

- An early HETGS (= HETG + ACIS-S) observation
- A recent HETGS (= HETG + ACIS-S) observation (with effective area reduced due to contamination)
- A random LETGS (=LETG + HRC-S) observation

Other modes are in use, e.g. LETG+ACIS-S, but this should be good enough to give a typical range of Chandra's capabilities. I downloaded the data from TGCat and will use teh arfs and rmfs to make the plot.

## XMM
I just chose to read the data from an XMM observation that I happened to have on file.

## Lynx
I'm using data from my most recent Lynx run for the CATXGS, covering XXX. 


In [ ]:
import os
import glob

import numpy as np
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.modeling import models, fitting

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
u.set_enabled_equivalencies(u.spectral())

## Read-in data

In [ ]:
def process_rmf(rmffile, path=''):
    rmf = Table.read(path+rmffile)
    # Assuming the entire peak in in the rmf (sometimes cut off near the edges)
    rmf['fwhm'] = [2*np.sqrt(2*(np.log(2))) * np.std(np.arange(len(row)) * row) for row in rmf['MATRIX']]
    rmf['wave'] = (0.5 * rmf['ENERG_LO'] + 0.5 * rmf['ENERG_HI']).to(u.Angstrom)
    # Assuming bins are about constant in wavelength range (I know they are not)
    rmf['delta_wave'] = rmf['ENERG_LO'].to(u.Angstrom) - rmf['ENERG_HI'].to(u.Angstrom)
    rmf['R'] = rmf['wave'] / (rmf['fwhm'] * rmf['delta_wave'])
    return rmf
    
def read_arf_rmf(rmffile, arfs, path=''):
    rmf = process_rmf(rmffile, path)
    # assuming arf and rmf have same energy grid
    return Table({'wave': rmf['wave'], 'R': rmf['R'], 'Aeff': sum([Table.read(path+a)['SPECRESP'] for a in arfs])})

def fwhm_from_fit(rmf):
    fit_g = fitting.LevMarLSQFitter()
    fwhm = np.zeros(len(rmf))
    for i in range(len(fwhm)):
        y = rmf['MATRIX'][i]
        x = np.arange(len(y))
        ind = y > 0
        g_init = models.Gaussian1D(amplitude=np.max(y), mean=np.argmax(y), stddev=20.)
        g = fit_g(g_init, x[ind], y[ind])
        fwhm[i] = 2*np.sqrt(2*(np.log(2))) * g.stddev
    return fwhm

In [ ]:
# Chandra
path = '/melkor/d1/guenther/downdata/Chandra/Lynxcompplot/tgcat/'
megearly = read_arf_rmf('meg_1.rmf.gz', ['meg_-1.arf.gz', 'meg_1.arf.gz'], path + 'obs_1236_tgid_3631/')
hegearly = read_arf_rmf('heg_1.rmf.gz', ['heg_-1.arf.gz', 'heg_1.arf.gz'], path + 'obs_1236_tgid_3631/')
meglate = read_arf_rmf('meg_1.rmf.gz', ['meg_-1.arf.gz', 'meg_1.arf.gz'], path + 'obs_20154_tgid_5483/')
heglate = read_arf_rmf('heg_1.rmf.gz', ['heg_-1.arf.gz', 'heg_1.arf.gz'], path + 'obs_20154_tgid_5483/')
heglate = read_arf_rmf('leg_1.rmf.gz', ['leg_-1.arf.gz', 'leg_1.arf.gz'], path + 'obs_20712_tgid_5404/')

# Cur off part where matrix does not contain complete RMF and thus the simple std fails
hegearly = hegearly[150:]

In [ ]:
path = '/melkor/d1/guenther/downdata/XMM/TWHya/0112880201/pps/'
xmmr1 = process_rmf('P0112880201R1S004RSPMAT1003.FTZ', path=path)
xmmr1['SPECRESP'] = [np.sum(row) for row in xmmr1['MATRIX']]
xmmr2 = process_rmf('P0112880201R2S005RSPMAT1003.FTZ', path=path)
xmmr2['SPECRESP'] = [np.sum(row) for row in xmmr2['MATRIX']]

In [ ]:
xmmr1['fwhm'] = fwhm_from_fit(xmmr1)

In [ ]:
xmmr2['fwhm'] = fwhm_from_fit(xmmr2)

In [ ]:
for rmf in [xmmr1, xmmr2]:
    rmf['R'] = rmf['wave'] / (rmf['fwhm'] * rmf['delta_wave'])

In [ ]:
xmm = xmmr1.copy()
xmm['SPECRESP'] = xmmr1['SPECRESP'] + xmmr2['SPECRESP']
xmm['Aeff'] = xmm['SPECRESP']
xmm['R'] = np.where(xmmr1['R'] > xmmr2['R'], xmmr1['R'], xmmr2['R'])
from scipy.signal import convolve, boxcar
#xmm['R'] = convolve(xmm['R'], boxcar(M=150), mode='same')

In [ ]:
plt.plot(xmm['wave'], xmm['R'], 'k')

def RunningPercentile(x,N):
    idx = np.arange(N) + np.arange(len(x)-N+1)[:,None]
    b = [row[row>0] for row in x[idx]]
    #return np.array(map(np.median,b))
    return np.array([np.percentile(c, 20) for c in b])

plt.plot(xmm['wave'][49: -50], RunningPercentile(xmm['R'], 100))

xmm['R'][49: -50] =  RunningPercentile(xmm['R'], 100)
# Cut off part where matrix does not contain complete RMF and thus the simple std fails
xmm = xmm[350:-350]

In [ ]:
lynx = Table.read('/melkor/d1/guenther/projects/Lynx/figures/lynxRaeff.ecsv', format='ascii')

In [ ]:
megearly.colnames

In [ ]:
plt.style.use('seaborn')
#plt.style.use('bmh')

In [ ]:
kwlynx = {'c': 'k', 'lw': 3, 'label': 'Lynx/XGS'}
kwxmm = {'ls': '--', 'label': 'XMM/RGS'}
kwchan = {'label': 'Chandra/MEG\n(at launch)'}

kwsinglefig = {'figsize': (5, 4)}
figpath = '/melkor/d1/guenther/projects/Lynx/figures/Jessica/'

In [ ]:
def allax(ax):
    ax.set_xlim(8, 40)
    ax.set_xlabel('wavelength [$\AA$]')    

def Aeff(ax):
    allax(ax)
    ax.plot(lynx['wave'] * 10, lynx['Aeff'], **kwlynx)
    ax.plot(xmm['wave'], xmm['Aeff'], **kwxmm)
    ax.plot(megearly['wave'], megearly['Aeff'], **kwchan)
    ax.set_ylabel('Effective area [cm$^2$]')
    ax.set_title('Effective area')

    
def R(ax):
    allax(ax)
    ax.plot(lynx['wave'] * 10, lynx['R'], **kwlynx)
    ax.plot(xmm['wave'], xmm['R'], **kwxmm)
    ax.plot(megearly['wave'], megearly['R'], **kwchan)
    ax.set_ylabel('Resolving power')
    ax.set_title('Resolving power')


def plot_FOM(ax):
    allax(ax)
    ax.plot(lynx['wave'] * 10, FOM(lynx), **kwlynx)
    ax.plot(xmm['wave'], FOM(xmm), **kwxmm)
    ax.plot(megearly['wave'], FOM(megearly), **kwchan)
    ax.set_title('FOM = $R * \sqrt{A_{\mathrm{eff}}}$')    
    ax.set_ylabel('FOM')

In [ ]:
def FOM(tab):
    return tab['R'] * np.sqrt(tab['Aeff'])

In [ ]:
fig, ax = plt.subplots(**kwsinglefig)
Aeff(ax)
#ax.legend(loc=(.4, .15))
fig.savefig(figpath + 'Aeff.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(**kwsinglefig)
Aeff(ax)
#ax.legend(loc=(.2, .6), ncol=2)
ax.set_yscale('log')
ax.set_ylim(1, 5e3)
fig.savefig(figpath + 'Aeff_log.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(**kwsinglefig)
R(ax)
#ax.legend(loc=(.4, .15))
ax.set_ylim(0, 1.5e4)
fig.savefig(figpath + 'R.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(**kwsinglefig)
R(ax)
#ax.legend(loc=(.4, .15))
ax.set_yscale('log')
ax.set_ylim(1e2, 3e4)
ax.set_ylim(0, 1.5e4)
fig.savefig(figpath + 'R_log.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(**kwsinglefig)
plot_FOM(ax)
#ax.legend(loc=(.4, .15))
fig.savefig(figpath + 'FOM.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(**kwsinglefig)
plot_FOM(ax)
#ax.legend(loc=(.4, .55))
ax.set_yscale('log')
ax.set_ylim(1e2, 1e6)
fig.savefig(figpath + 'FOM_log.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(**kwsinglefig)
plot_FOM(ax)
ax.set_yscale('log')
ax.set_ylim(1e2, 1e6)
ax.plot(hegearly['wave'], FOM(hegearly), label='Chandra/HEG')
ax.legend(loc=(.4, .55))

In [ ]:
fig, axes = plt.subplots(figsize=(12,3), ncols=3)

ax = axes[0]
ax.plot(lynx['wave'] * 10, lynx['Aeff'], **kwlynx)
ax.plot(xmm['wave'], xmm['Aeff'], **kwxmm)
ax.plot(megearly['wave'], megearly['Aeff'], **kwchan)

ax.set_xlim(8, 40)
ax.set_xlabel('wavelength [$\AA$]')
ax.set_ylabel('Effective area [cm$^2$]')
ax.set_yscale('log')
ax.set_ylim(1, 6e3)

ax.set_title('effective area')

ax=axes[1]
ax.plot(lynx['wave'] * 10, lynx['R'], **kwlynx)
ax.plot(xmm['wave'], xmm['R'], **kwxmm)
ax.plot(megearly['wave'], megearly['R'], **kwchan)

ax.set_xlim(8, 40)
ax.set_xlabel('wavelength [$\AA$]')
ax.set_ylabel('Resolving power')
ax.set_yscale('log')
ax.set_ylim(.5, 2e4)
ax.set_title('Resolving power')
ax.legend(loc='lower left', ncol=2)

ax = axes[2]
ax.plot(lynx['wave'] * 10, FOM(lynx), **kwlynx)
ax.plot(xmm['wave'], FOM(xmm), **kwxmm)
ax.plot(megearly['wave'], FOM(megearly), **kwchan)

ax.set_xlim(8, 40)
ax.set_xlabel('wavelength [$\AA$]')
ax.set_ylabel('FOM')
ax.set_yscale('log')
ax.set_ylim(50, 1e6)
ax.set_title('FOM = $R * \sqrt{A_{\mathrm{eff}}}$')
fig.subplots_adjust(wspace=.4)

In [ ]:
fig, axes = plt.subplots(figsize=(12,3), ncols=3)

ax = axes[0]
ax.plot(lynx['wave'] * 10, lynx['Aeff'], **kwlynx)
ax.plot(xmm['wave'], xmm['Aeff'], **kwxmm)
ax.plot(megearly['wave'], megearly['Aeff'], **kwchan)

ax.set_xlim(8, 40)
ax.set_xlabel('wavelength [$\AA$]')
ax.set_ylabel('Effective area [cm$^2$]')
ax.set_ylim(0, 5e3)

ax.set_title('Effective area')

ax=axes[1]
ax.plot(lynx['wave'] * 10, lynx['R'], **kwlynx)
ax.plot(xmm['wave'], xmm['R'], **kwxmm)
ax.plot(megearly['wave'], megearly['R'], **kwchan)

ax.set_xlim(8, 40)
ax.set_xlabel('wavelength [$\AA$]')
ax.set_ylabel('Resolving power')
ax.set_ylim(0, 1.15e4)
ax.set_title('Resolving power')
ax.legend(loc='lower left', ncol=2)

ax = axes[2]
ax.plot(lynx['wave'] * 10, FOM(lynx), **kwlynx)
ax.plot(xmm['wave'], FOM(xmm), **kwxmm)
ax.plot(megearly['wave'], FOM(megearly), **kwchan)

ax.set_xlim(8, 40)
ax.set_xlabel('wavelength [$\AA$]')
ax.set_ylabel('FOM')
ax.set_ylim(0, 6e5)
ax.set_title('FOM = $R * \sqrt{A_{\mathrm{eff}}}$')
fig.subplots_adjust(wspace=.4)